In [22]:
import os
# Import pandas
import pandas as pd
from datetime import datetime, timedelta

In [2]:
os.makedirs("../data", exist_ok=True)
!wget -qO ../data/customer_data.csv "https://assets.datacamp.com/production/repositories/1646/datasets/c3a701a4729471ae0b92d8c300b470fd2ec0a73a/user_demographics_v1.csv"
!wget -qO ../data/inapp_purchases.csv "https://assets.datacamp.com/production/repositories/1646/datasets/5decd183ef3710475958bbc903160fd6354379d5/purchase_data_v1.csv"

In [4]:
# Load the customer_data
customer_data = pd.read_csv('../data/customer_data.csv')
# Print the columns of customer data
print(customer_data.columns)
# Load the app_purchases
app_purchases = pd.read_csv('../data/inapp_purchases.csv')
# Print the columns of app_purchases
print(app_purchases.columns)

Index(['uid', 'reg_date', 'device', 'gender', 'country', 'age'], dtype='object')
Index(['date', 'uid', 'sku', 'price'], dtype='object')


In [6]:
# Merge on the 'uid' field
uid_combined_data = app_purchases.merge(customer_data, on=['uid'], how='inner')

# Examine the results 
print("number of record", len(uid_combined_data))
uid_combined_data.head()

number of record 9006


,date,uid,sku,price,reg_date,device,gender,country,age
0,2017-07-10,41195147,sku_three_499,499,2017-06-26T00:00:00Z,and,M,BRA,17
1,2017-07-15,41195147,sku_three_499,499,2017-06-26T00:00:00Z,and,M,BRA,17
2,2017-11-12,41195147,sku_four_599,599,2017-06-26T00:00:00Z,and,M,BRA,17
3,2017-09-26,91591874,sku_two_299,299,2017-01-05T00:00:00Z,and,M,TUR,17
4,2017-12-01,91591874,sku_four_599,599,2017-01-05T00:00:00Z,and,M,TUR,17


In [18]:
# Calculate the mean and median of price and age
purchase_summary = uid_combined_data.agg(
    {'price': ['mean', 'median'], 'age': ['mean', 'median']})
# Examine the output 
print(purchase_summary)

             price        age
mean    406.772596  23.922274
median  299.000000  21.000000


In [19]:
# Group the data 
grouped_purchase_data = uid_combined_data.groupby(
    by = ['device', 'gender'])
# Aggregate the data
purchase_summary = grouped_purchase_data.agg(
    {'price': ['mean', 'median', 'std']})
# Examine the results
print(purchase_summary)

                    price                   
                     mean median         std
device gender                               
and    F       400.747504    299  179.984378
       M       416.237308    499  195.001520
iOS    F       404.435330    299  181.524952
       M       405.272401    299  196.843197


In [20]:
# Calculate multiple metrics across different groups
grouped_purchase_data = uid_combined_data.groupby(by=['country', 'device'],
    axis=0, as_index=False)
grouped_purchase_data.agg({'price':['mean', 'min', 'max'],
    'age':['mean', 'min', 'max']})

country device       price                 age        
                         mean min  max       mean min max
0      BRA    and  412.985594  99  899  23.913565  15  62
1      BRA    iOS  404.739300  99  899  23.975681  15  66
2      CAN    and  406.826087  99  899  23.282609  15  49
3      CAN    iOS  386.573964  99  899  25.023669  15  57
4      DEU    and  402.474903  99  899  23.814672  15  66
5      DEU    iOS  417.639798  99  899  20.773300  15  42
6      FRA    and  418.377163  99  899  23.200692  15  55
7      FRA    iOS  382.921569  99  899  24.407843  15  59
8      TUR    and  433.913793  99  899  24.004310  15  56
9      TUR    iOS  390.176471  99  899  22.591176  15  51
10     USA    and  399.000000  99  899  24.564738  15  65
11     USA    iOS  410.684783  99  899  24.144022  15  75

In [21]:
def truncated_mean(data):
    """Compute the mean excluding outliers"""
    top_val = data.quantile(.9)
    bot_val = data.quantile(.1)
    trunc_data = data[(data<=top_val)&(data>=bot_val)]
    mean = trunc_data.mean()
    return(mean)
# Find the truncated mean age by group
grouped_purchase_data.agg({'age': [truncated_mean]})

,country,device,age
,,,truncated_mean
0,BRA,and,22.798977
1,BRA,iOS,22.784946
2,CAN,and,23.335000
3,CAN,iOS,24.048780
4,DEU,and,21.640257
5,DEU,iOS,19.611413
6,FRA,and,21.786885
7,FRA,iOS,23.123457
8,TUR,and,22.422572


In [23]:
# Restrict users by lapse date
current_date = pd.to_datetime('2018-03-17')
print(current_date)
# Maximum lapse date
print(grouped_purchase_data.lapse_date.max())
# Latest lapse date: a week before today
max_lapse_date = current_date - timedelta(days=7)
# Restrict to users lapsed before max_lapse_date
conv_sub_date = sub_data_demo[(sub_data_demo.lapse_date < max_lapse_date)]
# Count the users remaining in our data
total_users_count = count_sub_data.price.count()
print(total_users_count)

2018-03-17 00:00:00


AttributeError: 'DataFrameGroupBy' object has no attribute 'lapse_date'

In [ ]:
# Restrict subscription date
# Latest subscription date: within 7 days of lapsing
max_sub_date = conv_sub_data.lapse_date + timedelta(days=7)
# Filter the users with non-zero subscription price
# who subscribed before max_sub_date
total_subs = conv_sub_data[
    (conv_sub_data.price > 0)&
    (conv_sub_data.subscription_date<=max_sub_date)]